## HEPROW/UMG Post-Processing Tools

Tools to rip and plot modeled and unfolded neutron spectra.

First, import required modules and functions:

In [13]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

# Path to support scripts 
sys.path.insert(0,os.path.abspath('/home/pyne-user/Dropbox/UCB/Computational_Tools/Scripts/Python/Support'))

from DataManipulation import binIntegration, normAUBC
from DataIO import readDelimitedDataFile
from Plotting import Histogram

### User Inputs
#### Specify the file locations and type of data


HEPROW Unfolded spectrum:

In [5]:
#heprowPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Data/Simulated/88Inch_PHS/16MeVBe/SimulatedSmeared/"
heprowPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Data/Experiments/16MeVBe/Unfold/"

#outName="m16d_grv_out.flu"
#outName="m16d_mik_out.flu"
#heprowBinBounds="mid"

heprowName="mik.gru"
heprowBinBounds="low"
heprowHeaderLines=3
heprowDataStop=" *********format(16i5)*********"

NSD Unfolded Spectrum:

In [ ]:
nsdPath=heprowPath

nsdName="16BeSmearedSimNSDUnfold_50.txt"
nsdBinBounds="mid"
nsdHeaderLines=0

Simulated Spectrum:

In [8]:
#simPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Data/Simulated/88Inch_PHS/16MeVBe/SimulatedSmeared/Inputs/"
simPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Data/Experiments/16MeVBe/Unfold/Inputs/"

simName="inputSpectrum_50.txt"
simBinBounds="mid"
simHeaderLines=0

### Read in and Manipulate Data

Convert Each Data Set to a Probability Density Function (PDF)

In [14]:
# HEPROW
heprowData=readDelimitedDataFile(heprowPath+heprowName,header=heprowHeaderLines,breakText=heprowDataStop)

# Calculate the relative uncertainty
heprowData[2]=(np.asarray(heprowData[2])/np.asarray(heprowData[1])).tolist()

# Convert the unfolded spectrum to flux and normalize
heprowData[1]=binIntegration(heprowData[0],heprowData[1],edgeLoc=heprowBinBounds)
heprowData[1]=normAUBC(heprowData[1])

# Calculate the absolute error
heprowData[2]=(np.asarray(heprowData[2])*np.asarray(heprowData[1])).tolist()

# Build up histogram data
heprowHisto=Histogram()
heprowHisto.buildHisto(heprowData[0],heprowData[1],uncert=heprowData[2],edgeLoc=heprowBinBounds)
#print str(heprowHisto)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in divide


In [ ]:
# NSD Unfolding Algorithm
nsdData=readDelimitedDataFile(nsdPath+nsdName,header=nsdHeaderLines)

# Normalize the unfolded spectrum
nsdData[1]=normAUBC(nsdData[1])

# Build up histogram data
nsdHisto=Histogram()
nsdHisto.buildHisto(nsdData[0],nsdData[1],edgeLoc=nsdBinBounds)

In [9]:
# The simulated expected spectrum
simData=readDelimitedDataFile(simPath+simName,header=simHeaderLines)

# Convert the unfolded spectrum to flux and normalize
simData[1]=normAUBC(simData[1])

# Build up histogram data
simHisto=Histogram()
simHisto.buildHisto(simData[0],simData[1],edgeLoc=simBinBounds)

### Plot

In [15]:
# Allow use of Tex sybols
plt.rc('text', usetex=True)

# Set up figure
fig = plt.figure()
#ax1 = fig.add_subplot(111)
ax1 = fig.add_axes([0.1, 0.1, 0.8, 0.85])

# Preset data set format scheme
s=10
linewidth=['2.5']
marker=['.','o','v','^','<','>','1','2','3','4','8','s','p','*','h','H','+','x','d','D']
linestyle=['-','--',':']
dashes=[[10, 0.1],[10, 5, 10, 5],[0.25, 0.25, 0.25, 0.25]]

# Set Line color cycle
ax1.set_color_cycle(['k', 'k', 'k'])

# Set axes
ax1.axis([0, 20, 0, 1.25*max(heprowHisto.yValues)])
#ax1.set_xscale('log')
#ax1.set_yscale('log')

# Set axes labels and plot title.
ax1.set_title('\\textbf{16MeV D Breakup on Be - Smeared Simulation}', fontsize=18, weight="bold")    
ax1.set_xlabel('\\textbf{Energy [MeV]}', fontsize=18, weight="bold")
ax1.set_ylabel('\\textbf{Neutron PDF}', fontsize=18, weight="bold")
ax1.tick_params(axis='both', which='major', labelsize=18, width=2)
ax1.tick_params(axis='both', which='minor', width=2)

# Add data set to plot
ax1.errorbar(heprowHisto.midPtX, heprowHisto.midPtY, yerr=heprowHisto.sigma, marker=None, linestyle='None')
ax1.plot(heprowHisto.xEdges, heprowHisto.yValues, linewidth=linewidth[0], linestyle=linestyle[0], 
         marker=None,label="HEPROW Unfolded", dashes=dashes[0]) 
#ax1.plot(nsdHisto.xEdges, nsdHisto.yValues, linewidth=linewidth[0], linestyle=linestyle[1], 
#         marker=None,label="NSD Unfolded", dashes=dashes[1]) 
ax1.plot(simHisto.xEdges, simHisto.yValues, linewidth=linewidth[0], linestyle=linestyle[2], 
         label="GEANT Simulated") 


# Add and locate legend
leg = ax1.legend()
plt.legend(borderaxespad=0.75, loc=1, fontsize=16, handlelength=5, borderpad=0.5,\
            labelspacing=0.75, fancybox=True, framealpha=0.5, numpoints=1);

plt.show()

In [16]:
# Save the figure
fig.savefig(heprowPath+'Targ0Cut_PHS_25_smearedResp_25_1_results.png', bbox_inches='tight')